##[Compétition Kaggle : Web Traffic Time Series Forecasting](https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/43795)

## Perceptron

In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

import os
import sys

import torchvision.transforms.functional as TF
from torchvision import transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/E4/Projet\ Seq2seq

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/E4/Projet Seq2seq


In [0]:
class CSVDataset(Dataset):

    def __init__(self,file_path, n_x, n_y):
        super(CSVDataset,self).__init__()
        self.n_x = n_x
        self.n_y = n_y
        self.data = pd.read_csv(file_path,nrows=100)
        self.data = self.data.fillna(0)
        self.data = self.data[self.data.columns[1:]].values

    def __len__(self):
        return (self.data.shape[0]-self.n_x-self.n_y)*self.data.shape[1]

    def __getitem__(self, index):
        line = index //  (self.data.shape[1]-self.n_x-self.n_y)
        i = self.n_x + self.n_y + (index % (self.data.shape[1]-self.n_x-self.n_y))

        item = (torch.FloatTensor(self.data[line,i-self.n_y-self.n_x:i-self.n_y]),
                torch.FloatTensor(self.data[line,i-self.n_y:i]))
        item = (torch.stack([item[0]]),torch.stack([item[1]]))
        return item

In [0]:
n_x = 50 
n_y = 10 #données suivantes
train_dataset = CSVDataset("web-traffic-time-series-forecasting/train_1.csv", n_x, n_y)

In [0]:
trainloader = DataLoader(train_dataset, batch_size=256)

In [0]:
# class Feedforward(torch.nn.Module):
#         def __init__(self, input_size=1, hidden_size=10, output_size=1):
#             super(Feedforward, self).__init__()
#             self.input_size = input_size
#             self.hidden_size  = hidden_size
#             self.output_size = output_size
#             self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
#             self.relu = torch.nn.ReLU()
#             self.fc2 = torch.nn.Linear(self.hidden_size, self.output_size)
#             self.sigmoid = torch.nn.Sigmoid()        

#         def forward(self, x):
#             hidden = self.fc1(x)
#             relu = self.relu(hidden)
#             output = self.fc2(relu)
#             output = self.sigmoid(output)
#             return output

#https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379
class Net(torch.nn.Module):
    def __init__(self, input_size=1, hidden_size=10, output_size=1):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(input_size, hidden_size)   # hidden layer
        self.predict = torch.nn.Linear(hidden_size, output_size)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x


model = Net(input_size = n_x, output_size = n_y).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

Net(
  (hidden): Linear(in_features=50, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=10, bias=True)
)


In [0]:
epochs = 5

for i in range(epochs):

    for i_batch, data in enumerate(trainloader):
        x = data[0].to(device)
        y = data[1].to(device)
        optimizer.zero_grad()

        y_pred = model(x)

        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()

        # losses.append(single_loss.clone().detach().cpu().numpy())

        if i_batch%100 == 0:
            print(f'Loss at {i_batch}')

            
        
    if i%1 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

Loss at 0
epoch:   0 loss: 619.01141357
Loss at 0
epoch:   1 loss: 618.63616943
Loss at 0
epoch:   2 loss: 618.67590332
Loss at 0
epoch:   3 loss: 619.27783203
Loss at 0
epoch:   4 loss: 619.24560547
epoch:   4 loss: 619.2456054688
